In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import os

In [48]:
os.chdir('/content/drive/My Drive/Project/')

In [49]:
import numpy as np
import string, re
from string import digits
from numpy import array, argmax, random, take
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt

In [50]:
def read_data(file_name='fra.txt'):
    data_path = f"{file_name}"
    df = pd.read_table(data_path, names=["source","target","comments"])
    df = df.drop(columns=['comments'],axis=1)
    return df

In [51]:
# df = pd.read_table('fra.txt',names=["source","target","comments"])

In [52]:
df = read_data()

In [53]:
df

,source,target
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !
...,...,...
152578,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
152579,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
152580,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
152581,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [54]:
print(df.shape)

(152583, 2)


In [55]:
df = df[:75000]

In [56]:
df.shape

(75000, 2)

In [57]:
df.head()

,source,target
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !


In [58]:
df

,source,target
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !
...,...,...
74995,This hat is too big for you.,Ce chapeau est trop grand pour toi.
74996,This hat is too big for you.,Ce chapeau est trop grand pour vous.
74997,This house has eleven rooms.,Cette maison compte onze pièces.
74998,This house is far too small.,Cette maison est bien trop petite.


In [59]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
def preprocess(df):
  df.source = df.source.apply(lambda x: x.lower())
  df.traget = df.target.apply(lambda x: x.lower())

  # remove puncturation
  df.source = df.source.apply(lambda x: re.sub(r"([?.!])", r"\1 ",x))
  df.target = df.target.apply(lambda x: re.sub(r"([?.!])", r"\1 ",x))

  #
  df.source = df.source.apply(lambda x: re.sub(r'([" "])', " ",x))
  df.target = df.target.apply(lambda x: re.sub(r'([" "])', " ",x))

  # retain (a-z, A-Z, ".", "?", "!", ",")
  df.source = df.source.apply(lambda x: re.sub(r"[^a-zA-Z?.,!]+", " ",x))
  df.target = df.target.apply(lambda x: re.sub(r"[^a-zA-Z?.,!]+", " ",x))

  df.source = df.source.apply(lambda x: x.strip())
  df.target = df.target.apply(lambda x: x.strip())

  df.source = df.source.apply(lambda x: re.sub(" +", " ", x))
  df.target = df.target.apply(lambda x: re.sub(" +", " ", x))

  df.source = df.source.apply(lambda x: 'START_' + x + '_END')

  return df

In [61]:
df = preprocess(df)

In [62]:
df.head()

,source,target
0,START_go._END,Va !
1,START_run!_END,Cours !
2,START_run!_END,Courez !
3,START_wow!_END,a alors !
4,START_fire!_END,Au feu !


In [63]:
def tokenization(data, maxlen=100):
  token = Tokenizer(lower = True, oov_token='token')
  token.fit_on_texts(data)

  data_seq = token.texts_to_sequences(data)
  data_pad = pad_sequences(data_seq, maxlen=maxlen, padding='post')

  return token, data_pad

In [64]:
token_src, tensor_src = tokenization(df['source'], 50)
token_target, tensor_target = tokenization(df['target'], 50)

In [65]:
token_src

In [70]:
# tensor_src

In [71]:
# tensor_src.shape